In [1]:
import sys
sys.path.append('../Automatic-Circuit-Discovery/')
sys.path.append('..')

from acdc.greaterthan.utils import get_all_greaterthan_things
from ACDCPPExperiment import ACDCPPExperiment
from utils.prune_utils import get_3_caches, split_layers_and_heads
from transformer_lens import HookedTransformer

import numpy as np
import torch as t
import tqdm.notebook as tqdm
import json

import matplotlib.pyplot as plt

device = t.device('cuda') if t.cuda.is_available() else t.device('cpu')
print(f'Device: {device}')

/Users/canrager/.pyenv/versions/acdc_env/envs/acdcpp_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Device: cpu


# Greaterthan
## Model setup

In [2]:
model = HookedTransformer.from_pretrained(
    'gpt2-small',
    center_writing_weights=False,
    center_unembed=False,
    fold_ln=False,
    device=device,
)
model.set_use_hook_mlp_in(True)
model.set_use_split_qkv_input(True)
model.set_use_attn_result(True)

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-small into HookedTransformer


## Dataset setup

In [3]:
# Make clean dataset and reference dataset
N = 25

things = get_all_greaterthan_things(
    num_examples=N, metric_name="greaterthan", device=device
)
greaterthan_metric = things.validation_metric
clean_ds = things.validation_data # clean data x_i
corr_ds = things.validation_patch_data # corrupted data x_i'

print("\nClean dataset samples")
for i in range(5):
    print(model.tokenizer.decode(clean_ds[i]))

print("\nReference dataset samples")
for i in range(5):
    print(model.tokenizer.decode(corr_ds[i]))

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2 into HookedTransformer
Moving model to device:  cpu


Using pad_token, but it is not set yet.


Loaded pretrained model gpt2 into HookedTransformer
Moving model to device:  cpu

Clean dataset samples
The demonstrations lasted from the year 1267 to 12
The assaults lasted from the year 1644 to 16
The affair lasted from the year 1268 to 12
The stature lasted from the year 1653 to 16
The effort lasted from the year 1318 to 13

Reference dataset samples
The demonstrations lasted from the year 1201 to 12
The assaults lasted from the year 1601 to 16
The affair lasted from the year 1201 to 12
The stature lasted from the year 1601 to 16
The effort lasted from the year 1301 to 13


## Run combined ACDC++/ACDC experiment

In [4]:
RUN_NAME = "greaterthan_absval_edge"
acdcpp_greaterthan_convergence_thresh = 0.0004
acdc_paper_thresh = 0.015
num_thresholds = 5

ACDCPP_THRESHOLDS = np.linspace(
    0.75 * acdcpp_greaterthan_convergence_thresh,
    1.25 * acdcpp_greaterthan_convergence_thresh,
    num_thresholds
)
ACDC_THRESHOLDS = np.linspace(
    0.75 * acdc_paper_thresh,
    1.25 * acdc_paper_thresh,
    num_thresholds
)

In [ ]:
combined_exp = ACDCPPExperiment(
    model,
    clean_ds,
    corr_ds,
    acdc_metric=greaterthan_metric,
    acdcpp_metric=greaterthan_metric,
    acdc_thresholds=ACDC_THRESHOLDS,
    acdcpp_thresholds=ACDCPP_THRESHOLDS,
    run_name=RUN_NAME,
    verbose=False,
    attr_absolute_val=True,
    save_graphs_after=0,
    pruning_mode='edge',
    no_pruned_nodes_attr=1,
)
pruned_heads, num_passes, pruned_attrs = combined_exp.run()